In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
file_name = 'KPMG_cleaned.xlsx'
print(pd.ExcelFile(file_name).sheet_names)

['Title Sheet', 'Transactions', 'CustomerDemographic', 'CustomerAddress', 'NewCustomerList']


In [3]:
tran_df = pd.read_excel(file_name, sheet_name='Transactions')
tran_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,profit,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,17.87,2012-12-02 00:00:00
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,1702.55,2014-03-03 00:00:00
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1544.61,1999-07-20 00:00:00
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,817.36,1998-12-16 00:00:00
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,1055.82,2015-08-10 00:00:00


In [35]:
np.shape(tran_df)

(20000, 14)

In [36]:
tran_df.isna().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
profit                     730
product_first_sold_date    197
dtype: int64

In [37]:
tran_df.duplicated().sum()

0

In [38]:
tran_clean_df = tran_df.dropna()

In [39]:
tran_clean_df.isna().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
profit                     0
product_first_sold_date    0
dtype: int64

In [40]:
np.shape(tran_clean_df)

(19270, 14)

In [41]:
tran_clean_df['transaction_date'].describe()

<ipython-input-41-004e09bb4de3>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  tran_clean_df['transaction_date'].describe()


count                   19270
unique                    364
top       2017-08-18 00:00:00
freq                       81
first     2017-01-01 00:00:00
last      2017-12-30 00:00:00
Name: transaction_date, dtype: object

In [42]:
most_recent_purchase = tran_clean_df['transaction_date'].max()
tran_clean_df['last_purchase_days_ago'] = most_recent_purchase - tran_clean_df['transaction_date']
tran_clean_df['last_purchase_days_ago'] /= np.timedelta64(1,'D')
tran_clean_df.head()

<ipython-input-42-ac13fee99579>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tran_clean_df['last_purchase_days_ago'] = most_recent_purchase - tran_clean_df['transaction_date']
<ipython-input-42-ac13fee99579>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tran_clean_df['last_purchase_days_ago'] /= np.timedelta64(1,'D')


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,profit,product_first_sold_date,last_purchase_days_ago
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,17.87,2012-12-02 00:00:00,308.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,1702.55,2014-03-03 00:00:00,223.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1544.61,1999-07-20 00:00:00,75.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,817.36,1998-12-16 00:00:00,121.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,1055.82,2015-08-10 00:00:00,90.0


In [43]:
tran_clean_df[tran_clean_df['last_purchase_days_ago']>365].shape

(0, 15)

In [44]:
rfm_table = tran_clean_df.groupby('customer_id').agg({
    'last_purchase_days_ago': lambda x: x.min(),
    'customer_id': lambda x: len(x),
    'profit': lambda x: x.sum()
})

rfm_table.rename(columns= {'last_purchase_days_ago':'recency','customer_id':'frequency','profit':'monetary'},
                 inplace=True)


In [45]:
rfm_table.head()

,recency,frequency,monetary
customer_id,,,
1,7.0,11,3018.09
2,128.0,3,2226.26
3,102.0,7,3290.21
4,195.0,2,220.57
5,16.0,6,2394.94


In [46]:
rfm_table.shape

(3489, 3)

In [47]:
quartiles = rfm_table.quantile(q=[0.25,0.50,0.75])
quartiles

,recency,frequency,monetary
0.25,18.0,4.0,1779.70
0.50,45.0,5.0,2785.04
0.75,88.0,7.0,4095.63


In [48]:
def Rscore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    if x <= d[p][0.50]:
        return 3
    if x <= d[p][0.75]:
        return 2
    else: 
        return 1
    
def FMscore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    if x <= d[p][0.50]:
        return 2
    if x <= d[p][0.75]:
        return 3
    else: 
        return 4  

In [49]:
rfmseg = rfm_table 
rfmseg['r_score'] = rfmseg['recency'].apply(Rscore, args=('recency', quartiles))
rfmseg['f_score'] = rfmseg['frequency'].apply(FMscore, args=('frequency',quartiles))
rfmseg['m_score'] = rfmseg['monetary'].apply(FMscore, args=('monetary',quartiles))
rfmseg.head()

,recency,frequency,monetary,r_score,f_score,m_score
customer_id,,,,,,
1,7.0,11,3018.09,4,4,3
2,128.0,3,2226.26,1,1,2
3,102.0,7,3290.21,1,3,3
4,195.0,2,220.57,1,1,1
5,16.0,6,2394.94,4,3,2


In [50]:
rfmseg['rfm_value'] = 100 * rfmseg['r_score'] + 10 * rfmseg['f_score'] + rfmseg['m_score']
rfmseg.head()

,recency,frequency,monetary,r_score,f_score,m_score,rfm_value
customer_id,,,,,,,
1,7.0,11,3018.09,4,4,3,443
2,128.0,3,2226.26,1,1,2,112
3,102.0,7,3290.21,1,3,3,133
4,195.0,2,220.57,1,1,1,111
5,16.0,6,2394.94,4,3,2,432


In [55]:
rfm_quartiles = rfmseg['rfm_value'].quantile(q=[0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90])
rfm_quartiles.head()

0.1    112.0
0.2    132.0
0.3    212.0
0.4    233.0
0.5    311.0
Name: rfm_value, dtype: float64

In [56]:
rfm_quartiles

0.1    112.0
0.2    132.0
0.3    212.0
0.4    233.0
0.5    311.0
0.6    324.0
0.7    344.0
0.8    413.0
0.9    433.0
Name: rfm_value, dtype: float64

In [57]:
def rfmclass(x,p,d):
    if x <= d[0.10]:
        return 'lost customer'
    if x <= d[0.20]:
        return 'almost lost customer'
    if x <= d[0.30]:
        return 'high risk customer'
    if x <= d[0.40]:
        return 'losing customer'
    if x <= d[0.50]:
        return 'late bloomer'
    if x <= d[0.60]:
        return 'potential customer'
    if x <= d[0.70]:
        return 'recent customer'
    if x <= d[0.80]:
        return 'becoming loyal'
    if x <= d[0.90]:
        return 'very loyal'
    else:
        return 'platinum customer'

In [58]:
rfm_title = rfm_table
rfm_title.head()

,recency,frequency,monetary,r_score,f_score,m_score,rfm_value
customer_id,,,,,,,
1,7.0,11,3018.09,4,4,3,443
2,128.0,3,2226.26,1,1,2,112
3,102.0,7,3290.21,1,3,3,133
4,195.0,2,220.57,1,1,1,111
5,16.0,6,2394.94,4,3,2,432


In [59]:
rfm_title['customer_title'] = rfm_title['rfm_value'].apply(rfmclass, args=('rfm_value',rfm_quartiles))
rfm_title.head()

,recency,frequency,monetary,r_score,f_score,m_score,rfm_value,customer_title
customer_id,,,,,,,,
1,7.0,11,3018.09,4,4,3,443,platinum customer
2,128.0,3,2226.26,1,1,2,112,lost customer
3,102.0,7,3290.21,1,3,3,133,high risk customer
4,195.0,2,220.57,1,1,1,111,lost customer
5,16.0,6,2394.94,4,3,2,432,very loyal


In [60]:
cust_seg = rfm_title.reset_index()
cust_seg.head()

,customer_id,recency,frequency,monetary,r_score,f_score,m_score,rfm_value,customer_title
0,1,7.0,11,3018.09,4,4,3,443,platinum customer
1,2,128.0,3,2226.26,1,1,2,112,lost customer
2,3,102.0,7,3290.21,1,3,3,133,high risk customer
3,4,195.0,2,220.57,1,1,1,111,lost customer
4,5,16.0,6,2394.94,4,3,2,432,very loyal


In [61]:
cust_seg.to_csv('rfm.csv', index=False)